<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/notebooks/diksha/analyse_LA_mobility_whitepop_time_series_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [2]:
# add time information at the end of every cell
!pip install ipython-autotime
%load_ext autotime

  Using cached https://files.pythonhosted.org/packages/b4/c9/b413a24f759641bc27ef98c144b590023c8038dfb8a3f09e713e9dff12c1/ipython_autotime-0.3.1-py2.py3-none-any.whl
time: 2.03 ms (started: 2021-04-16 04:32:03 +00:00)


In [3]:
# Install required dependancies
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 67kB/s 
     |████████████████████████████████| 204kB 31.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=bd21c4e7c5ec59b8179087df00ba1e207e83488e4f8cd2430b5b0e088bfa50ea
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [4]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 104 ms (started: 2021-04-16 04:33:03 +00:00)


In [5]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 311 ms (started: 2021-04-16 04:33:03 +00:00)


In [6]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 5.6 s (started: 2021-04-16 04:33:03 +00:00)


In [7]:
#sc.stop()

time: 1.51 ms (started: 2021-04-16 04:33:09 +00:00)


In [8]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 31.7 s (started: 2021-04-16 04:33:09 +00:00)


In [9]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'la'

time: 1.79 ms (started: 2021-04-16 04:33:40 +00:00)


In [10]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [11]:
# View schema
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [12]:
# # Take small sample of data to experiment with
# df_soc = df_soc.limit(100)
# df_soc.show()

time: 1.12 ms (started: 2021-04-16 04:34:04 +00:00)


In [13]:
df_soc.createOrReplaceTempView('clean_la')
df_soc = spark.sql('SELECT cbg, date_range_start as date, device_count, completely_home_device_count, part_time_work_behavior_devices, full_time_work_behavior_devices FROM clean_la WHERE device_count > 5')
df_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|060371831013|2020-02-07T00:00:...|          45|                           6|                              1|                              1|
|060374034022|2020-02-07T00:00:...|         155|                          24|                             12|                              4|
|060375326041|2020-02-07T00:00:...|          31|                           4|                              6|                              2|
|060375544031|2020-02-07T00:00:...|         130|                          27|                             16|                             10|
|06037

In [14]:
# View schema
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)

time: 4.36 ms (started: 2021-04-16 04:34:05 +00:00)


In [15]:
# Add completely home percentage column
df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('part_time_work_percentage', (df_soc['part_time_work_behavior_devices']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

df_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|completely_home_percentage|part_time_work_percentage|full_time_work_percentage|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+
|060371831013|2020-02-07T00:00:...|          45|                           6|                              1|                              1|        13.333333333333334|       2.2222222222222223|       2.2222222222222223|
|060374034022|2020-02-07T00:00:...|         155|                          24|                             12|       

In [16]:
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- completely_home_percentage: double (nullable = true)
 |-- part_time_work_percentage: double (nullable = true)
 |-- full_time_work_percentage: double (nullable = true)

time: 20.6 ms (started: 2021-04-16 04:34:05 +00:00)


In [17]:
#Create temp view
df_soc.createOrReplaceTempView('mobility')

time: 56.8 ms (started: 2021-04-16 04:34:05 +00:00)




---


**Demographic Data**






---



In [18]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'la.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|060373104003|     1286|17.884914463452567| 69.51788491446345|
|060590865023|     2545|22.789783889980352| 85.34381139489194|
|060376510024|     1285|               0.0|42.490272373540854|
|060376511011|     2708| 7.745398773006134| 50.70162481536189|
|060590994023|      672|12.969283276450511| 38.54166666666667|
|060590995094|      342| 17.83625730994152| 77.48538011695906|
|060376511012|     2483|2.6178010471204187|50.302053966975436|
|060376512011|      859|2.8846153846153846| 82.42142025611176|
|060376512012|     1179| 1.441899915182358|  74.1306191687871|
|060376512014|      989|2.4266936299292214| 70.57633973710819|
|060376512211|     3373|  9.95684340320592| 40.61666172546695|
|060590995092|      938| 5.756929637526652|  84.9680170575693|
|060590995095|      576|14.583333333333334|           7

In [19]:
df_mob_demo = spark.sql('SELECT mobility.*, demographic.perc_whiteonly from mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()


+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|completely_home_percentage|part_time_work_percentage|full_time_work_percentage|    perc_whiteonly|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+------------------+
|060371831013|2020-02-07T00:00:...|          45|                           6|                              1|                              1|        13.333333333333334|       2.2222222222222223|       2.2222222222222223| 71.91906805640711|
|060374034022|2020-02-07T00:00:...|     

In [20]:
# DIVIDING THE PEOPLE TOP 20 PERCENT AND BOTTOM 20 PERCENT

df_mob_demo_high_class =  spark.sql('SELECT * FROM demographic_mobility WHERE perc_whiteonly > 80')
grouped_df_mob_demo_high_class = df_mob_demo_high_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_high_class.createOrReplaceTempView('mob_demo_white_majority')


df_mob_demo_low_class =  spark.sql('SELECT * FROM demographic_mobility WHERE perc_whiteonly < 20')
grouped_df_mob_demo_low_class = df_mob_demo_low_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_low_class.createOrReplaceTempView('mob_demo_white_minority')



grouped_df_mob_demo_combined = spark.sql("""

SELECT wh.date,wh.completely_home_percentage as high_comp_home_perc,
wl.completely_home_percentage as low_comp_home_perc  
FROM mob_demo_white_majority wh INNER JOIN mob_demo_white_minority wl ON wh.date = wl.date

""")


time: 426 ms (started: 2021-04-16 04:34:08 +00:00)


In [21]:
#grouped_df_mob_demo_combined.show()


time: 927 µs (started: 2021-04-16 04:34:08 +00:00)


In [22]:
grouped_df_mob_demo_combined_pandas = grouped_df_mob_demo_combined.toPandas()
grouped_df_mob_demo_combined_pandas

,date,high_comp_home_perc,low_comp_home_perc
0,2020-07-02T00:00:00-07:00,33.024489,41.119156
1,2020-11-13T00:00:00-08:00,27.363667,33.195375
2,2020-06-02T00:00:00-07:00,40.422056,46.124976
3,2020-06-04T00:00:00-07:00,34.011674,39.214923
4,2020-10-29T00:00:00-07:00,27.431963,32.490376
...,...,...,...
361,2020-04-28T00:00:00-07:00,42.465187,44.830043
362,2020-05-26T00:00:00-07:00,37.081699,41.830156
363,2020-06-13T00:00:00-07:00,32.776078,38.444930
364,2020-08-28T00:00:00-07:00,29.625158,36.695802


time: 5min 51s (started: 2021-04-16 04:34:08 +00:00)


---
**Graph** 

---

In [23]:
base = alt.Chart(grouped_df_mob_demo_combined_pandas.reset_index()).encode(x='date')

alt.layer(
    base.mark_line(color='blue').encode(y='high_comp_home_perc'),
    
    base.mark_line(color='red').encode(y='low_comp_home_perc')

)

alt.LayerChart(...)

time: 145 ms (started: 2021-04-16 04:40:00 +00:00)
